KINGS COUNTY HOUSE SALES ANALYSIS.

Please fill out:
* Student names:
1. Benson Kamau 
2. Kelvin Muchori
3. Nancy Chelagat
4. Breden Mugambi
5. Sally Kinyanjui

* Student pace: Full time

* Scheduled project review date/time: N/A

* Instructor name: Nikita Njoroge


BUSINESS UNDERSTANDING

This study aims to explore the relationship between key factors such as renovations, bedrooms, and other property attributes with housing prices in King County. By leveraging on the dataset available, we seek to uncover the specific impact of renovations on property valuations, considering variables such as the extent of renovations.

Additionally, we aim to investigate how the number of bedrooms, bathrooms, along with other structural features like square footage and lot size, view  influences housing prices.
This can help homeowners to add functionality and beauty to their property while simultaneously boosting its resale value.

Main Objective

To build and evaluate models using various combinations of the available features in the King County.


Specific Objectives

1.	What features of a home increase its value in King County?
2.	Which features increase a home’s value the most in King County?
3.	To evaluate which combinations of the available features in the dataset are the most impactful features for predicting sale price.


DATA UNDERSTANDING

This project analyzes data about homes sold in King County, Washington between May 2014 and May 2015 in order to make recommendations to relevant stakeholders.

This dataset is housed in the kc_house_data.csv file within the project's data folder and the columns outlined in the accompanying column_names.md file.


In [8]:
#import the relevant library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm

# Define the DataFrame in the global scope
df = pd.DataFrame()

#create a function that loads data and gets the info about the data.
def load_and_get_info(file_path):
    """
    Load data from a CSV file and get information about the DataFrame.
    
    Parameters:
    - file_path (str): Path to the CSV file.
    
    Returns:
    - df_info (str): Information about the DataFrame.
    """
    # Load data
    global df
    df = pd.read_csv(file_path)
    
    # Get information about the DataFrame
    df_info = df.info()
    
    return df_info

file_path = 'data\kc_house_data.csv'  
data_info = load_and_get_info(file_path)
print(data_info)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

This dataset contains records of 21,597 home sales. It includes:

How much the house sold for

Square footage of the house's living space

Square footage of the lot

Number of floors, bedrooms, and bathrooms

The quality of the view from the house 

Whether the house is on a waterfront

The house's grade (relates to construction materials and quality)

The house's condition (relates to age and maintenance)

The year the house was built

The year the house was renovated

The location of the house



In [11]:
def check_dtypes(df):
    """
    Check data types and identify the kind of variable for each column in the DataFrame.
    
    Parameters:
    - df (DataFrame): Input DataFrame.
    
    Returns:
    - dtypes_info (DataFrame): DataFrame containing information about data types and variable types.
    """
    # Get data types of each column
    dtypes = df.dtypes
    
    # Identify variable types (numeric or categorical) for each column
    variable_types = []
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            variable_types.append('Numeric')
        else:
            variable_types.append('Categorical')
    
    # Create DataFrame to store information
    dtypes_info = pd.DataFrame({'Column': dtypes.index, 'Data Type': dtypes.values, 'Variable Type': variable_types})
    
    return dtypes_info

data_types_info = check_dtypes(df)
print(data_types_info)

           Column Data Type Variable Type
0              id     int64       Numeric
1            date    object   Categorical
2           price   float64       Numeric
3        bedrooms     int64       Numeric
4       bathrooms   float64       Numeric
5     sqft_living     int64       Numeric
6        sqft_lot     int64       Numeric
7          floors   float64       Numeric
8      waterfront    object   Categorical
9            view    object   Categorical
10      condition    object   Categorical
11          grade    object   Categorical
12     sqft_above     int64       Numeric
13  sqft_basement    object   Categorical
14       yr_built     int64       Numeric
15   yr_renovated   float64       Numeric
16        zipcode     int64       Numeric
17            lat   float64       Numeric
18           long   float64       Numeric
19  sqft_living15     int64       Numeric
20     sqft_lot15     int64       Numeric
